# surplus 2.0.0 playground notebook

wrangling with environments for devbox users using codium/vs code:

```text
$ devbox shell                    # enter devbox env
(surplus-py3.11) (devbox) $ exit  # leave poetry env
(devbox) $ codium .               # open ide
```

In [1]:
# converting nominatim keys to OUTPUT_LINE_X_KEYS format

keys = """
region, county, state, state_district, country, continent
"""

split_keys = [f'"{key.strip()}"' for key in keys.strip().split(",")]

print(f"OUTPUT_LINE_X_KEYS: Final[tuple[str, ...]] = ({','.join(split_keys)},)")

OUTPUT_LINE_X_KEYS: Final[tuple[str, ...]] = ("region","county","state","state_district","country","continent",)


In [1]:
from surplus import PlusCodeQuery, LocalCodeQuery, LatlongQuery, StringQuery
from surplus import Latlong, Result
from surplus import default_geocoder, default_reverser

## Generic Result NamedTuple

In [2]:
nom_result = Result[int](3)

try:
    1 / 0
except Exception as exc:
    exc_result = Result[int](-1, error=exc)

print("{}\t{:<40}\t{}".format(bool(nom_result), repr(nom_result.error), nom_result.get()))
print(
    "{}\t{:<40}\t{}".format(
        bool(exc_result), repr(exc_result.error), exc_result.cry(string=True)
    )
)
print("{}\t{:<40}\t{}".format(bool(exc_result), repr(exc_result.error), exc_result.get()))

True	None                                    	3
False	ZeroDivisionError('division by zero')   	division by zero (ZeroDivisionError)


ZeroDivisionError: division by zero

## Query Types

In [3]:
PlusCodeQuery(code="6PH58QMF+FV").to_lat_long_coord(geocoder=default_geocoder)

Result(value=Latlong(latitude=1.3336875, longitude=103.7746875), error=None)

In [4]:
plus_code = LocalCodeQuery(code="8QMF+FV", locality="Singapore").to_full_plus_code(
    geocoder=default_geocoder
)

PlusCodeQuery(code=plus_code.get()).to_lat_long_coord(geocoder=default_geocoder)

Result(value=Latlong(latitude=1.3336875, longitude=103.7746875), error=None)

In [5]:
LocalCodeQuery(code="8QMF+FV", locality="Singapore").to_lat_long_coord(
    geocoder=default_geocoder
)

Result(value=Latlong(latitude=1.3336875, longitude=103.7746875), error=None)

In [6]:
LatlongQuery(
    latlong=Latlong(latitude=1.33318835, longitude=103.77461234638255)
).to_lat_long_coord(geocoder=default_geocoder)

Result(value=Latlong(latitude=1.33318835, longitude=103.77461234638255), error=None)

In [7]:
StringQuery(query="Ngee Ann Polytechnic").to_lat_long_coord(geocoder=default_geocoder)

Result(value=Latlong(latitude=1.33318835, longitude=103.77461234638255), error=None)

## return dictionary of `reverser` function

all the necessary keys (see `SHAREABLE_TEXT_LINE_*` constants) should be at the top-level of the dictionary.
these keys will be casted into strings for safety guarantee when shareable text lines are being generated.

while not necessary, consider keeping the original response dict under the "raw" key.
helps with debugging using `-d/--debug`!

In [ ]:
reverser_return = {
    "amenity": "Ngee Ann Polytechnic",
    "house_number": "535",
    "road": "Clementi Road",
    "suburb": "Bukit Timah",
    "city": "Singapore",
    "county": "Northwest",
    "ISO3166-2-lvl6": "SG-03",
    "postcode": "599489",
    "country": "Singapore",
    "country_code": "sg",
    "raw": {
        "place_id": 297946059,
        "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
        "osm_type": "relation",
        "osm_id": 2535118,
        "lat": "1.33318835",
        "lon": "103.77461234638255",
        "class": "amenity",
        "type": "university",
        "place_rank": 30,
        "importance": 0.34662169301918117,
        "addresstype": "amenity",
        "name": "Ngee Ann Polytechnic",
        "display_name": "Ngee Ann Polytechnic, 535, Clementi Road, Bukit Timah, Singapore, Northwest, 599489, Singapore",
        "address": {
            "amenity": "Ngee Ann Polytechnic",
            "house_number": "535",
            "road": "Clementi Road",
            "suburb": "Bukit Timah",
            "city": "Singapore",
            "county": "Northwest",
            "ISO3166-2-lvl6": "SG-03",
            "postcode": "599489",
            "country": "Singapore",
            "country_code": "sg",
        },
        "boundingbox": ["1.3289692", "1.3372184", "103.7701481", "103.7783945"],
    },
    "latitude": 1.33318835,
    "longitude": 103.77461234638255,
}

In [7]:
import pprint

latlong = LocalCodeQuery(code="8QMF+FV", locality="Singapore").to_lat_long_coord(
    default_geocoder
)
if not latlong:
    latlong.cry()

else:
    location = default_reverser(latlong.get())
    pprint.pprint(location)

{'ISO3166-2-lvl6': 'SG-03',
 'amenity': 'Ngee Ann Polytechnic',
 'city': 'Singapore',
 'country': 'Singapore',
 'country_code': 'sg',
 'county': 'Northwest',
 'house_number': '535',
 'latitude': 1.33318835,
 'longitude': 103.77461234638255,
 'postcode': '599489',
 'raw': {'address': {'ISO3166-2-lvl6': 'SG-03',
                     'amenity': 'Ngee Ann Polytechnic',
                     'city': 'Singapore',
                     'country': 'Singapore',
                     'country_code': 'sg',
                     'county': 'Northwest',
                     'house_number': '535',
                     'postcode': '599489',
                     'road': 'Clementi Road',
                     'suburb': 'Bukit Timah'},
         'addresstype': 'amenity',
         'boundingbox': ['1.3289692',
                         '1.3372184',
                         '103.7701481',
                         '103.7783945'],
         'class': 'amenity',
         'display_name': 'Ngee Ann Polytechnic, 535, Clem